In [1]:
!pip install tensorflow-datasets

In [2]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\affinity\anaconda3\lib\site-packages (19.3.1)


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import load_model
from datetime import datetime
from keras import regularizers
#tf.enable_eager_execution()

Using TensorFlow backend.


In [4]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [5]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_contrast(image, lower=2.0, upper = 4.0)
    image = tf.image.random_hue(image, max_delta = 0.8)
    image = tf.image.random_saturation(image, lower=2.0, upper=4.0)
    return image, label

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, 3, kernel_regularizer=regularizers.l2(0.001), activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, 3, kernel_regularizer=regularizers.l2(0.001),activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(256, 3,kernel_regularizer=regularizers.l2(0.001), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(256, 3,kernel_regularizer=regularizers.l2(0.001), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(256, 3, kernel_regularizer=regularizers.l2(0.001),activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(512, 3,kernel_regularizer=regularizers.l2(0.001), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(512, 3, kernel_regularizer=regularizers.l2(0.001),activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 256)       1024      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       590080    
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 256)       1

In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).batch(64) 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [9]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_accuracy',
    save_best_only=True)

In [10]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [11]:
history = model.fit(train_data,
                    epochs=100,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

Epoch 0 ends at 10:04:40.042274: {'loss': 3.8071762252807617, 'accuracy': 0.421025, 'val_loss': 3.303182456144102, 'val_accuracy': 0.3731}
Epoch 1 ends at 10:05:42.004615: {'loss': 3.1255492015838624, 'accuracy': 0.5197, 'val_loss': 3.817743650667227, 'val_accuracy': 0.2738}
Epoch 2 ends at 10:06:52.270734: {'loss': 2.6038786796569826, 'accuracy': 0.5904, 'val_loss': 3.3474865308992423, 'val_accuracy': 0.3262}
Epoch 3 ends at 10:08:00.622981: {'loss': 2.453968006324768, 'accuracy': 0.621175, 'val_loss': 2.934883524657814, 'val_accuracy': 0.4361}
Epoch 4 ends at 10:09:07.691226: {'loss': 2.420680487251282, 'accuracy': 0.641775, 'val_loss': 2.997026519410929, 'val_accuracy': 0.4183}
Epoch 5 ends at 10:10:16.562954: {'loss': 2.4406861267089845, 'accuracy': 0.652, 'val_loss': 3.097807302596463, 'val_accuracy': 0.4603}
Epoch 6 ends at 10:11:25.727692: {'loss': 2.3174808221817016, 'accuracy': 0.672325, 'val_loss': 2.363996654559093, 'val_accuracy': 0.6045}
Epoch 7 ends at 10:12:34.500068: {'

Epoch 58 ends at 11:17:15.829559: {'loss': 0.9477567176818847, 'accuracy': 0.848625, 'val_loss': 1.120626088160618, 'val_accuracy': 0.7984}
Epoch 59 ends at 11:18:34.866593: {'loss': 0.9564913302421569, 'accuracy': 0.847275, 'val_loss': 1.1431157573772843, 'val_accuracy': 0.7888}
Epoch 60 ends at 11:19:55.358901: {'loss': 0.9446400838851928, 'accuracy': 0.850975, 'val_loss': 1.0840496382895548, 'val_accuracy': 0.8077}
Epoch 61 ends at 11:21:13.899789: {'loss': 0.948911387348175, 'accuracy': 0.850575, 'val_loss': 1.3409168462085117, 'val_accuracy': 0.736}
Epoch 62 ends at 11:22:31.837257: {'loss': 0.9385342192649841, 'accuracy': 0.85145, 'val_loss': 1.2868560799367867, 'val_accuracy': 0.7434}
Epoch 63 ends at 11:23:51.399341: {'loss': 0.942238445854187, 'accuracy': 0.84945, 'val_loss': 1.10017920755277, 'val_accuracy': 0.7939}
Epoch 64 ends at 11:25:12.186048: {'loss': 0.9335017906188965, 'accuracy': 0.852675, 'val_loss': 1.1759325041892423, 'val_accuracy': 0.7834}
Epoch 65 ends at 11:2

In [12]:
model.evaluate(test_data)

157/157 [==============================]1.0683 - accuracy: 0.765 - 0s 110ms/step - loss: 1.0348 - accuracy: 0.781 - 0s 93ms/step - loss: 0.9620 - accuracy: 0.812 - 0s 87ms/step - loss: 0.9346 - accuracy: 0.82 - 0s 80ms/step - loss: 0.9195 - accuracy: 0.82 - 0s 76ms/step - loss: 0.9300 - accuracy: 0.81 - 1s 72ms/step - loss: 0.9509 - accuracy: 0.80 - 1s 69ms/step - loss: 0.9623 - accuracy: 0.80 - 1s 67ms/step - loss: 0.9595 - accuracy: 0.81 - 1s 65ms/step - loss: 0.9772 - accuracy: 0.80 - 1s 64ms/step - loss: 0.9585 - accuracy: 0.81 - 1s 63ms/step - loss: 0.9764 - accuracy: 0.81 - 1s 62ms/step - loss: 0.9751 - accuracy: 0.81 - 1s 61ms/step - loss: 0.9704 - accuracy: 0.81 - 1s 60ms/step - loss: 0.9723 - accuracy: 0.81 - 1s 60ms/step - loss: 0.9790 - accuracy: 0.81 - 1s 59ms/step - loss: 0.9862 - accuracy: 0.81 - 1s 58ms/step - loss: 0.9938 - accuracy: 0.81 - 1s 58ms/step - loss: 0.9893 - accuracy: 0.81 - 1s 58ms/step - loss: 0.9954 - accuracy: 0.81 - 1s 57ms/step - loss: 0.9850 - accurac

[1.0316843682793295, 0.814]

In [13]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))